In [ ]:
from huggingface_hub import login
login("your_access_token")

In [37]:
import tensorflow as tf
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import numpy as np

In [38]:
dataset = load_dataset("charles828/vertex-ai-customer-support-training-dataset")

dataset

DatasetDict({
    train: Dataset({
        features: ['flags', 'instruction', 'category', 'intent', 'response', 'text'],
        num_rows: 24184
    })
    test: Dataset({
        features: ['flags', 'instruction', 'category', 'intent', 'response', 'text'],
        num_rows: 2688
    })
})

In [39]:
def preprocess_data(example):
    # Menggabungkan kolom instruction dan text untuk input, dan response untuk label
    input_text = example["instruction"] + " " + example["text"]
    response = example["response"]
    return {"input_text": input_text, "response": response}


dataset = dataset.map(preprocess_data)


In [40]:
train_test_split = dataset["train"].train_test_split(test_size=0.1, seed=42)
train_data = train_test_split["train"]
val_data = train_test_split["test"]
test_data = dataset["test"]


In [ ]:
model_name = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token="your_access_token")
model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token="your_access_token")


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

d:\Python\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\models--meta-llama--Llama-3.2-1B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

d:\Python\Lib\site-packages\huggingface_hub\file_download.py:653: UserWarning: Not enough free disk space to download the file. The expected file size is: 2471.65 MB. The target location C:\Users\USER\.cache\huggingface\hub\models--meta-llama--Llama-3.2-1B\blobs only has 160.18 MB free disk space.
  warnings.warn(


ConnectionError: (MaxRetryError('HTTPSConnectionPool(host=\'cdn-lfs-us-1.hf.co\', port=443): Max retries exceeded with url: /repos/46/00/46001ae0478831903029d3dfca642974a652dfe8c5a4a525a48a3c700a7a99dd/68a2e4be76fa709455a60272fba8e512c02d81c46e6c671cc9449e374fd6809a?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1732765606&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMjc2NTYwNn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzQ2LzAwLzQ2MDAxYWUwNDc4ODMxOTAzMDI5ZDNkZmNhNjQyOTc0YTY1MmRmZThjNWE0YTUyNWE0OGEzYzcwMGE3YTk5ZGQvNjhhMmU0YmU3NmZhNzA5NDU1YTYwMjcyZmJhOGU1MTJjMDJkODFjNDZlNmM2NzFjYzk0NDllMzc0ZmQ2ODA5YT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=N7ofAsah3e1cUy~C3ENja3KRZnmJo5KiD10IRlIYibt7u-dij~iTi6xfvZ-g9rEvKw0bSi6-jutoKu~-zikMRX~OwVZjd501-lJb~IT-n5mb3qsQNzkDBEQFLiMmZkDoTHv6jokb~AUL2FNGCFTRSvPgPSuuaHCNiYVwNssDWNCPIhfg83u899PAU~wZDyw

In [ ]:
max_seq_length = 128

# Fungsi untuk tokenisasi data
def tokenize_function(examples):
    inputs = tokenizer(
        examples["input_text"], max_length=max_seq_length, truncation=True, padding="max_length", return_tensors="tf"
    )
    outputs = tokenizer(
        examples["response"], max_length=max_seq_length, truncation=True, padding="max_length", return_tensors="tf"
    )
    inputs["labels"] = outputs["input_ids"]
    return inputs


train_data = train_data.map(tokenize_function, batched=True)

NameError: name 'train_data' is not defined

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_data),
    train_data["labels"]
)).shuffle(1000).batch(8)

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_data),
    val_data["labels"]
)).batch(8)

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


NameError: name 'train_data' is not defined

In [ ]:
model.compile(optimizer=optimizer, loss=loss)


NameError: name 'model' is not defined

In [ ]:
model.fit(train_dataset, validation_data=val_dataset, epochs=3)


NameError: name 'model' is not defined

In [ ]:
def chat_with_model(input_text):
    inputs = tokenizer(input_text, return_tensors="tf", truncation=True, max_length=max_seq_length, padding="max_length")
    outputs = model.generate(**inputs, max_new_tokens=50)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Uji model dengan pertanyaan
test_text = "How can I reset my password?"
response = chat_with_model(test_text)
print("User:", test_text)
print("Bot:", response)